In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recidivism-forecasting-challenge/NIJ_s_Recidivism_Challenge_Test_Dataset1.csv
/kaggle/input/recidivism-forecasting-challenge/NIJ_s_Recidivism_Challenge_Training_Dataset.csv
/kaggle/input/recidivism-forecasting-challenge/submission_sample.csv


In [2]:
#loading the dataset
test = pd.read_csv("/kaggle/input/recidivism-forecasting-challenge/NIJ_s_Recidivism_Challenge_Test_Dataset1.csv")
train = pd.read_csv("/kaggle/input/recidivism-forecasting-challenge/NIJ_s_Recidivism_Challenge_Training_Dataset.csv")
submission_sample = pd.read_csv("/kaggle/input/recidivism-forecasting-challenge/submission_sample.csv")

In [3]:
test['Recidivism_Arrest_Year1'] = np.nan

In [4]:
train = train[test.columns]

In [5]:
df = pd.concat([train,test]).reset_index(drop=True)

In [6]:
df.isnull().sum()/len(df)

ID                                                   0.000000
Gender                                               0.000000
Race                                                 0.000000
Age_at_Release                                       0.000000
Residence_PUMA                                       0.000000
Gang_Affiliated                                      0.122586
Supervision_Risk_Score_First                         0.018386
Supervision_Level_First                              0.066576
Education_Level                                      0.000000
Dependents                                           0.000000
Prison_Offense                                       0.126843
Prison_Years                                         0.000000
Prior_Arrest_Episodes_Felony                         0.000000
Prior_Arrest_Episodes_Misd                           0.000000
Prior_Arrest_Episodes_Violent                        0.000000
Prior_Arrest_Episodes_Property                       0.000000
Prior_Ar

In [7]:
df["Gang_Affiliated"] = df.Gang_Affiliated.fillna(-99)
df["Supervision_Risk_Score_First"] = df["Supervision_Risk_Score_First"].fillna(df["Supervision_Risk_Score_First"].mean())
df.loc[df["Supervision_Level_First"].isnull(), "Supervision_Level_First"] = "Others"
df["Prison_Offense"] = df["Prison_Offense"].fillna(-99)

In [8]:
df.ID.dtype

dtype('int64')

In [9]:
for column in df.columns:
   if df[column].dtype == "bool":
        df[column] = df[column].astype("int")

In [10]:
dummy_variables = [column for column in df.columns if df[column].dtype=="object"]

In [11]:
df1 = pd.get_dummies(df, columns = dummy_variables, drop_first = True)

In [12]:
train = df1[df1['Recidivism_Arrest_Year1'].notnull()]
test = df1[df1['Recidivism_Arrest_Year1'].isnull()].reset_index(drop=True)

In [13]:
X = train.drop(["ID", "Recidivism_Arrest_Year1"], axis=1)
y = train["Recidivism_Arrest_Year1"]

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X,y)

RandomForestClassifier()

In [15]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid 
param_grid = {
    'bootstrap': [True],
    'max_features': [2, 3],
    'n_estimators': [200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [16]:
grid_search.fit(X,y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_features': [2, 3],
                         'n_estimators': [200, 300, 1000]},
             verbose=2)

In [17]:
grid_search.best_estimator_

RandomForestClassifier(max_features=3, n_estimators=1000)

In [18]:
rf = RandomForestClassifier(max_features=3, n_estimators=1000)

In [19]:
rf.fit(X,y)

RandomForestClassifier(max_features=3, n_estimators=1000)

In [20]:
test = test.drop(["ID", "Recidivism_Arrest_Year1"], axis=1)

XGBoost

In [21]:
rf.predict_proba(test)

array([[0.787, 0.213],
       [0.676, 0.324],
       [0.75 , 0.25 ],
       ...,
       [0.806, 0.194],
       [0.918, 0.082],
       [0.919, 0.081]])

In [22]:
submission_sample["Recidivism_Arrest_Year1"] = rf.predict_proba(test)[:,1]

In [23]:
submission_sample["Recidivism_Arrest_Year1"]

0       0.213
1       0.324
2       0.250
3       0.295
4       0.206
        ...  
7802    0.114
7803    0.055
7804    0.194
7805    0.082
7806    0.081
Name: Recidivism_Arrest_Year1, Length: 7807, dtype: float64

In [24]:
submission_sample.to_csv("submission.csv", index = False)

In [25]:
predictions = submission_sample["Recidivism_Arrest_Year1"] 

XGBoost

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42, shuffle = True)

In [27]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=200)

xgb.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:12:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=200, n_jobs=2,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
prediction = xgb.predict_proba(X_test)[:,1]

In [29]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, prediction)

0.6601019478948791

In [30]:
param_test2b = {
    'min_child_weight':[6,10],
}


gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
                                                   subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                                   nthread=4, scale_pos_weight=1,seed=27), param_grid = param_test2b, 
                         scoring='roc_auc',n_jobs=4, cv=2, verbose = 2)
gsearch2b.fit(X_train,y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] END ...bootstrap=True, max_features=2, n_estimators=200; total time=   3.0s
[CV] END ...bootstrap=True, max_features=2, n_estimators=200; total time=   3.0s
[CV] END ...bootstrap=True, max_features=2, n_estimators=300; total time=   4.6s
[CV] END ..bootstrap=True, max_features=2, n_estimators=1000; total time=  14.6s
[CV] END ..bootstrap=True, max_features=2, n_estimators=1000; total time=  15.2s
[CV] END ...bootstrap=True, max_features=3, n_estimators=300; total time=   4.7s
[CV] END ..bootstrap=True, max_features=3, n_estimators=1000; total time=  15.6s
[CV] END ..bootstrap=True, max_features=3, n_estimators=1000; total time=  11.4s
[18:12:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END ................................

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:12:59] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=4, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=140, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_

In [31]:
gsearch2b.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=10, missing=nan, monotone_constraints='()',
              n_estimators=140, n_jobs=4, nthread=4, num_parallel_tree=1,
              predictor='auto', random_state=27, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, seed=27, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [32]:
grid_search.best_score_

0.7058464900172389

In [33]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=4,
                    subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                    nthread=4, scale_pos_weight=1,seed=27, min_child_weight=10 )

In [34]:
xgb.fit(X_train,y_train)
prediction = xgb.predict_proba(X_test)[:,1]
roc_auc_score(y_test, prediction) 

[18:13:03] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7073988702494273

In [35]:
prediction  = xgb.predict_proba(test)[:,1]
submission_xgb = submission_sample.copy()
submission_xgb['Recidivism_Arrest_Year1'] = prediction

In [36]:
submission_xgb.to_csv("sumbission_xgb.csv", index = False)

In [37]:
submission_xgb

,ID,Recidivism_Arrest_Year1
0,6,0.155612
1,8,0.252294
2,12,0.208481
3,15,0.214675
4,16,0.135917
...,...,...
7802,26746,0.084786
7803,26747,0.102600
7804,26749,0.110156
7805,26752,0.110347


In [38]:
submission_sample ["Recidivism_Arrest_Year1"] = 0.3*submission_sample['Recidivism_Arrest_Year1'] + 0.7*submission_xgb["Recidivism_Arrest_Year1"]

In [39]:
submission_sample.to_csv('submission_blend.csv', index = False)